In [20]:
import numpy as np
import math as mt
import sympy as sp
from sympy.physics.quantum import TensorProduct as TP
from math import sqrt
import sys
import random

In [21]:
#Qubits iniciais
qbit0= np.matrix([1,0]).transpose()
qbit1= np.matrix([0,1]).transpose()


#Porta H
H = 1/sqrt(2)*(np.matrix([[1,1], [1,-1]]))

#Aplicação da porta Hadamard no qubit qx
Hqbit0= H*qbit0
#print(Hqbit0)

#Aplicação da porta CNOT
CNOT = np.matrix([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]])

#Base de Bell estado |00>

#Definição Alvo e Controle
AC= TP(Hqbit0,qbit0)
#print(AC)

Bell00 = CNOT * AC

#print(Bell00)

   

In [22]:
#Determinação do qubit para o protocolo
Message_alfa = float(input('Qual o estado alfa do Qubit que deseja enviar?'))
Message_beta = float(input('Qual o estado beta do Qubit que deseja enviar?'))


#Verifica se a soma é igual a 1 (condição de normalização)
if not mt.isclose(Message_alfa  + Message_beta, 1.0):
    print("Erro: A soma das entradas não é igual a 1.")
    sys.exit()
    
#Determina a mensagem a ser enviada
alfa_inicial = (Message_alfa * qbit0)
beta_inicial = (Message_beta * qbit1)
Message = alfa_inicial + beta_inicial

print(Message)

#Estado psi_0 

psi_0 = TP(qbit0, qbit0, qbit0) + TP(qbit0, qbit1, qbit1) + TP(qbit1,qbit0, qbit0) + TP(qbit1,qbit1,qbit1)

#Aplicação da porta CNOT
#Dimensionamento para 3 qubits
#Porta I
I = np.matrix ([[1,0], [0,1]])
CNOT_I = TP(CNOT,I)

#print(CNOT_I)
psi_1 = CNOT_I * psi_0

#print(psi_1)

#Aplicação da porta Hadamard
H_Message_alfa = H * alfa_inicial
H_Message_beta = H * beta_inicial

#print(H_Message_alfa, H_Message_beta)

Qual o estado alfa do Qubit que deseja enviar?0.5
Qual o estado beta do Qubit que deseja enviar?0.5
[[0.5]
 [0.5]]


In [23]:
#Medida e Reconstrução da Mensagem

estadoa0 = float(H_Message_alfa[0][0])
estadoa1 = float(H_Message_alfa[1][0])
estadob0 = float(H_Message_beta[0][0])
estadob1 = float(H_Message_beta[1][0])


#Determinação dos estados do qbitmensagem
a0 = np.matrix([estadoa0,0]).transpose()
a1 = np.matrix([0,estadoa1]).transpose()
b0 = np.matrix([estadob0,0]).transpose()
b1 = np.matrix([0,estadob1]).transpose()

print(a0,a1,b0,b1)

#Determinação dos possíveis estados da Mensagem

#Medição

estado_00 = TP(qbit0, qbit0)
estado_11 = TP(qbit1, qbit1)
estado_10 = TP(qbit1, qbit0)
estado_01 = TP(qbit0, qbit1)

group_estados = [estado_00,estado_11,estado_10,estado_01]
chosen_estados = random.choice(group_estados)
Medição = chosen_estados

del estado_00, estado_11, estado_10, estado_01

print(Medição)

[[0.35355339]
 [0.        ]] [[0.        ]
 [0.35355339]] [[0.35355339]
 [0.        ]] [[ 0.        ]
 [-0.35355339]]
[[0]
 [0]
 [1]
 [0]]


In [26]:
#Reconstrução da Mensagem
#Define o qubit mensagem em função do estado medido
if  np.all(Medição == (TP(qbit0,qbit0))):
    Message_tp = (a0-b1)
elif  np.all(Medição == (TP(qbit1,qbit1))):
    Message_tp = (a1-b0)
elif  np.all(Medição == (TP(qbit0,qbit1))):
    Message_tp = (a1+b0)
elif np.all(Medição == (TP(qbit1,qbit0))):
    Message_tp = (a0+b1)
#print(Message_tp) 


#Recupera a mensagem Original

# Define as matrizes de Pauli
X = np.matrix([[0, 1], [1, 0]])
Z = np.matrix([[1, 0], [0, -1]])

if np.all(Message_tp == (a0-b1)):
    Message_final = Message_tp
elif np.all(Message_tp == (a1-b0)):
    Message_final = Z * X * Message_tp 
elif np.all(Message_tp == (a1+b0)):
    Message_final = X * Message_tp 
elif np.all(Message_tp == (a0+b1)):
    Message_final = Z * Message_tp 
    
Mensagem_teletransportada = Message_final*sqrt(2)    

#print(Mensagem_teletransportada)

alfa = float(Mensagem_teletransportada[0][0])
beta = float(Mensagem_teletransportada[1][0])

alfa_final = np.matrix([alfa,0]).transpose()
beta_final = np.matrix([0,beta]).transpose()

#print(alfa_final, beta_final)

#Aplicação de ruído
#Determinando os ruídos
bitflip = np.matrix([[0, 1], [1, 0]])
phaseflip = np.matrix([[1, 0], [0, -1]])
noisebit = 0

# Escolha do ruído aplicado
while True:
    noise = input('Escolha o ruído:[0]- sem ruído, [1]- bitflip, [2]- phaseflip: ')
    if noise in ['0', '1', '2']:
        noise = int(noise)
        break
    else:
        print("Opção inválida. Por favor, digite 0, 1, 2 ou 3.")

if noise == 1:
    noisebit = bitflip
elif noise == 2:
    noisebit = phaseflip
elif noise == 0:
    noisebit = 1
    
#print(noisebit)

aplicação_noise_a = noisebit * alfa_final
aplicação_noise_b = noisebit * beta_final

#print(aplicação_noise_a, aplicação_noise_b)

alfa_noise = aplicação_noise_a
beta_noise = aplicação_noise_b

alfa_final = alfa_noise
beta_final = beta_noise
#print(alfa_final, beta_final)
#print(alfa_inicial[0][0],alfa_final[1][0],alfa_inicial[1][0],alfa_inicial[0][0])

if np.all((alfa_final == alfa_inicial) & (beta_final == beta_inicial)):
    print('Teletransporte concluído com sucesso e sem presença de ruídos')
else:
    if np.all((alfa_inicial[0][0] == alfa_final[1][0]) & (alfa_inicial[1][0] == alfa_final[0][0]) & (beta_inicial[1][0] == beta_final[0][0]) & (beta_inicial[0][0] == beta_final[1][0])):
        print("Mensagem teletransportada com ruído do tipo bitflip")
    elif np.all((alfa_inicial[0][0] == alfa_final[0][0]) & (alfa_inicial[1][0] == -alfa_final[1][0]) & (beta_inicial[1][0] == -beta_final[1][0]) &(beta_inicial[0][0] == beta_final[0][0])):
        print("Mensagem teletransportada com ruído do tipo phaseflip")
    else:
        print("Erro de teletransporte")






Escolha o ruído:[0]- sem ruído, [1]- bitflip, [2]- phaseflip: 2
Mensagem teletransportada com ruído do tipo phaseflip
